In [ ]:

#only to check if i can replicate the results

ecopg = EcologicalPublicGood(N=2, f=1.2, c=5, m=-4, qc=0.2, qr= 0.1, degraded_choice=True)

ecopg_with_history = HistoryEmbedded(ecopg, h = (1,1,1))

for mode in ['both_state_and_action_information', 'only_action_history_information', 'only_state_information', 'no_information']:
 
    # Initialize the information condition
    information_condition_instance = Information_Conditions(ecopg, mode=mode)
    mae = POstratAC(env=information_condition_instance, learning_rates=0.1, discount_factors=0.9)

    # Data storage
    
    avg_coop_time_pairs = []
    num_samples = 10
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    print(f"\nMode: {mode}")

    # Monte Carlo Simulations
    for initial_condition in initial_conditions_list:

        # initial_condition = make_degraded_state_cooperation_probablity_one(initial_condition, information_condition_instance.Oset[0]) #to make sure all of them start at the same position in the degraded state (shouldn't it)
        xtraj, fixedpointreached = mae.trajectory(initial_condition, Tmax=10000, tolerance=1e-5)
        final_point = xtraj[-1]
        
        avg_coop_across_states = get_average_cooperativeness_including_degraded_state(policy=final_point, obsdist=mae.obsdist(final_point), mode = mode, Oset = mae.env.Oset[0])[0]
        time_to_reach = xtraj.shape[0]

        # Store cooperativeness and time as pairs (round cooperativeness to 2 decimals)
        avg_coop_time_pairs.append((round(avg_coop_across_states, 2), time_to_reach))

    # Create DataFrame for processing
    df = pd.DataFrame(avg_coop_time_pairs, columns=["AverageCooperation", "TimeToReach"])
    total_count = len(df)
    
    average_cooperation_across_initial_conditions = np.round(df['AverageCooperation'].agg('mean'), 3)
    print("Mean Final Cooperation Across Initial Conditions ", average_cooperation_across_initial_conditions)


    # Add a classification column
    def classify(avg_coop):
        if avg_coop < 0.1:
            return "Defection"
        elif avg_coop > 0.9:
            return "Cooperation"
        else:
            return "Mixed"


    df['Classification'] = df['AverageCooperation'].apply(classify)
    average_cooperation_across_initial_conditions = df['AverageCooperation'].agg('mean')
    # Reporting unique entries

    # Overall Summary
    summary = df.groupby('Classification')['TimeToReach'].agg(
        MedianTimetoReach='median',
        Percentage= lambda x: round((len(x) / total_count) * 100,1)
    ).reset_index()
    
    print(summary)

'''
Output: 

Mode: both_state_and_action_information
Mean Final Cooperation Across Initial Conditions  0.88
  Classification  MedianTimetoReach  Percentage
0    Cooperation              102.0        63.6
1          Mixed              295.5        36.4

Mode: only_action_history_information
Mean Final Cooperation Across Initial Conditions  0.156
  Classification  MedianTimetoReach  Percentage
0    Cooperation               59.0         3.6
1      Defection             1842.0        60.0
2          Mixed             2856.0        36.4

Mode: only_state_information
Mean Final Cooperation Across Initial Conditions  0.85
  Classification  MedianTimetoReach  Percentage
0    Cooperation              101.5        54.5
1          Mixed              223.0        45.5

Mode: no_information
Mean Final Cooperation Across Initial Conditions  0.005
  Classification  MedianTimetoReach  Percentage
0      Defection             1832.0       100.0


----







'''

In [ ]:
def run_simulation(mae, information_condition_instance, num_samples, initial_cooperation_in_degraded_state, include_degraded_state_for_average_cooperation):
    """
    Runs Monte Carlo simulations and collects average cooperation and time-to-reach data.

    Parameters:
        mae: The POstratAC instance (learning agent).
        information_condition_instance: The instance of Information_Conditions.
        num_samples (int): Number of initial conditions to sample.
        initial_cooperation_in_degraded_state (int): If 0, cooperation in degraded state is set to zero; 
                                                     if 1, it is set to one; otherwise, no changes.
        include_degraded_state_for_average_cooperation (bool): Whether to include the degraded state in average cooperation.

    Returns:
        list: A list of (average cooperation, time-to-reach) tuples.
    """
    avg_coop_time_pairs = []
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    for initial_condition in initial_conditions_list:
        if initial_cooperation_in_degraded_state == 0:
            initial_condition = make_degraded_state_cooperation_probablity_zero(
                initial_condition, information_condition_instance.Oset[0]
            )
        elif initial_cooperation_in_degraded_state == 1:
            initial_condition = make_degraded_state_cooperation_probablity_one(
                initial_condition, information_condition_instance.Oset[0]
            )

        xtraj, fixedpointreached = mae.trajectory(initial_condition, Tmax=10000, tolerance=1e-5)
        final_point = xtraj[-1]

        if include_degraded_state_for_average_cooperation:  
                avg_coop_across_states = get_average_cooperativeness_excluding_degraded_state(
                    policy=final_point, 
                    obsdist=mae.obsdist(final_point), 
                    mode=mode, 
                    Oset=mae.env.Oset[0]
                )[0]

        else:
                avg_coop_across_states = get_average_cooperativeness_excluding_degraded_state(
                    policy=final_point, 
                    obsdist=mae.obsdist(final_point), 
                    mode=mode, 
                    Oset=mae.env.Oset[0]
                )[0]


        time_to_reach = xtraj.shape[0]

        avg_coop_time_pairs.append((round(avg_coop_across_states, 2), time_to_reach))

    return avg_coop_time_pairs

In [ ]:
"adjust initial condition degraded state"

def adjust_initial_condition_degraded_state(initial_condition, Oset, adjustment):

    if adjustment == "make_degraded_state_cooperation_probablity_zero":

        degraded_mask = jnp.array(['g' in label for label in Oset])
        initial_condition[:, degraded_mask, 0] = 0
        initial_condition[:, degraded_mask, 1] = 1

    if adjustment == "make_degraded_state_cooperation_probablity_one"

    return initial_condition

def make_degraded_state_cooperation_probablity_one(initial_condition, Oset):

    degraded_mask = jnp.array(['g' in label for label in Oset])
    initial_condition[:, degraded_mask, 0] = 1
    initial_condition[:, degraded_mask, 1] = 0

    return initial_condition



In [ ]:


# Store results for different m values
results = []

for m_value in range(0, -7, -2):  # Loop from 0 to -6
    print(f"\nRunning for m = {m_value}")

    # Initialize the ecological model for the current m
    ecopg = EcologicalPublicGood(N=2, f=1.2, c=5, m= m_value, qc=0.02, qr= 0.0001, degraded_choice=False)

    # Store cooperation basin sizes for different informational conditions
    cooperation_basin_sizes = {}

    for mode in ['both_state_and_action_information', 'only_action_history_information', 'only_state_information', 'no_information']:
        
        information_condition_instance = Information_Conditions(ecopg, mode=mode)
        mae = POstratAC(env=information_condition_instance, learning_rates=0.1, discount_factors=0.99)

        avg_coop_values = []
        num_samples = 5
        initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

        for initial_condition in initial_conditions_list:
            initial_condition = make_degraded_state_cooperation_probablity_zero(initial_condition, information_condition_instance.Oset[0]) #to make sure all of them start at the same position in the degraded state (shouldn't it)
            xtraj, _ = mae.trajectory(initial_condition, Tmax=10000, tolerance=1e-5)
            final_point = xtraj[-1]
            avg_coop = get_average_cooperativeness_excluding_degraded_state(policy=final_point, obsdist=mae.obsdist(final_point), mode=mode, Oset=mae.env.Oset[0])[0]
            avg_coop_values.append(round(avg_coop, 3))

        df = pd.DataFrame(avg_coop_values, columns=["AverageCooperation"])
        
        # Classification
        df['Classification'] = np.where(df['AverageCooperation'] > 0.9, "Cooperation", 
                                        np.where(df['AverageCooperation'] < 0.1, "Defection", "Mixed"))
    
        # Compute percentage of cooperation
        coop_basin_size = (df['Classification'] == 'Cooperation').mean() * 100
        cooperation_basin_sizes[mode] = round(coop_basin_size, 2)

    # Compute absolute increase relative to "no_information"
    baseline = cooperation_basin_sizes["no_information"]
    absolute_increase = {mode: round(size - baseline, 1) for mode, size in cooperation_basin_sizes.items()}

    # Store results in a structured format
    for mode in cooperation_basin_sizes:
        results.append({
            "m": m_value,
            "Information Condition": mode,
            "Cooperation Basin Size (%)": cooperation_basin_sizes[mode],
            "Absolute Increase (%)": absolute_increase[mode] if mode != "no_information" else 0
        })

# Convert results to DataFrame for easy viewing
df_results = pd.DataFrame(results)

# Display results

df_results


In [ ]:
'''avg policy for obs set'''


#| export

#average policy 

def average_policy_for_given_observation_set(X, O):
    """
    Takes in the strategy wrt to complete state set as input and returns the average strategy wrt to the given observation set. For example Strategty with respect to state set might
    have c, c, g = 0.2 and c, c, p = 0.8. But the observation set will be c, c, if only actions are observed. Then the average strategy for this observation set would be 0.5

    """
    A, S, F = X.shape        # Number of layers, states, and features
    _, _, num_obs = O.shape   # Number of observations

    # Initialize the output matrix with zeros
    output = np.zeros((A, num_obs, F))

    for i in range(A):
        for obs in range(num_obs):
            current_mask = O[i, :, obs]  # Shape: (S,)

            # Select rows from X where mask is 1
            selected_X = X[i][current_mask == 1]  # Shape: (num_selected_states, F)
            if selected_X.size > 0:
                mean_vector = selected_X.mean(axis=0)  # Shape: (F,)
            else:
                mean_vector = np.zeros(F)  # Default to zero vector
            output[i, obs, :] = mean_vector

    return output




In [ ]:
"""random discrepances observed"""


#degraded choice is false

ecopg = EcologicalPublicGood(N=2, f=1.2, c=5, m=-4, qc=0.02, qr= 0.0001, degraded_choice=False)
# ecopg = EcologicalPublicGood(N=2, f=1.2, c=5, m=-4, qc=0.2, qr= 0.1, degraded_choice=True)


for mode in ['both_state_and_action_information', 
             'only_action_history_information', 
             'only_state_information', 
             'no_information'
             ]:
# for mode in ['both_state_and_action_information']:

    # Initialize the information condition

    information_condition_instance = Information_Conditions(ecopg, mode=mode)
    mae = POstratAC(env=information_condition_instance, learning_rates=0.1, discount_factors=0.99)

    # Data storage
    
    avg_coop_time_pairs = []
    num_samples = 5
    initial_conditions_list = lhs_sampling(mae.Q, num_samples, mae.N)

    print(f"\nMode: {mode}")

    # Monte Carlo Simulations
    for initial_condition in initial_conditions_list:
    
        # initial_condition = make_degraded_state_cooperation_probablity_zero(initial_condition, information_condition_instance.Oset[0]) #to make sure all of them start at the same position in the degraded state (shouldn't it)

        xtraj, fixedpointreached = mae.trajectory(initial_condition, Tmax=10000, tolerance=1e-5)
        final_point = xtraj[-1]
        
        avg_coop_across_states = get_average_cooperativeness(policy=final_point, obsdist=mae.obsdist(final_point), mode = mode, Oset = mae.env.Oset[0], exclude_degraded_state_for_average_cooperation=True)[0]
        time_to_reach = xtraj.shape[0]

        avg_coop_time_pairs.append((avg_coop_across_states, time_to_reach))

    # Create DataFrame for processing
    df = pd.DataFrame(avg_coop_time_pairs, columns=["AverageCooperation", "TimeToReach"])
    total_count = len(df)
    # print(df)
    
    average_cooperation_across_initial_conditions = df['AverageCooperation'].agg('mean')
    print("Mean Final Cooperation Across Initial Conditions ", np.round(average_cooperation_across_initial_conditions,2))


    # Add a classification column
    def classify(avg_coop):
        if avg_coop < 0.1:
            return "Defection"
        elif avg_coop > 0.9:
            return "Cooperation"
        else:
            return "Mixed"


    df['Classification'] = df['AverageCooperation'].apply(classify)
    # Reporting unique entries
    # Overall Summary
    summary = df.groupby('Classification')['TimeToReach'].agg(
        MedianTimetoReach='median',
        Percentage= lambda x: round((len(x) / total_count) * 100,1)
    ).reset_index()
    
    print(summary)
